# Installation

With conda, you can install the required dependencies with:

```bash
conda install bs4 requests lxml
```


# Basic usage of BeautifulSoup

First, we import the `BeatifulSoup` class:

In [3]:
from bs4 import BeautifulSoup

We load the html source file from disk and pass the source the the BeautifulSoup constructor. We choose the "lxml" parser for XML documents, which is faster than the defaul parser that comes with BeautifulSoup:

In [4]:
src = open("list.html")
document = BeautifulSoup(src, 'lxml')
print(document)

<!DOCTYPE html>
<html>
<body>
<h2>An Unordered HTML List</h2>
<ul id="unordered_list" style="color:#069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ul>
<h2>An Ordered HTML List</h2>
<ol id="ordered_list" style="color:#069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ol>
</body>
</html>



### Finding tags by name

The document now contains the full html document. We can find the first occuring tag with a specific name with the `find` function. Let's find the first un-ordered list tag:

In [5]:
ulist = document.find("ul")

The result contains all tags contained in the matched tag:

In [6]:
ulist

<ul id="unordered_list" style="color:#069">
<li>Coffee</li>
<li>Tea</li>
<li>Milk</li>
</ul>

The `find_all` function returns **all** tags that match the given tag name. We can use it to get a list of all list items:

In [8]:
items = ulist.find_all("li")
items

[<li>Coffee</li>, <li>Tea</li>, <li>Milk</li>]

Finally, we can loop over all items and extract their contant with the `get_text` function:

In [12]:
for item in items:
    print(item.get_text())

Coffee
Tea
Milk


Note that `find_all` is **recursive** by default. This means that we could call it the on the full `document` to get the items
of both the ordered and un-ordered lists:

In [10]:
document.find_all("li", recursive=True)

[<li>Coffee</li>,
 <li>Tea</li>,
 <li>Milk</li>,
 <li>Coffee</li>,
 <li>Tea</li>,
 <li>Milk</li>]

### Finding tags by attributes

Sometimes the easiest way to find a tag is by its attribute name. In our examples, both lists have an `id` attribute that uniquely identifies the tables. We can also use the `find*` methods to search for attributes:


In [13]:
document.find_all(attrs={"style":"color:#069"})

[<ul id="unordered_list" style="color:#069">
 <li>Coffee</li>
 <li>Tea</li>
 <li>Milk</li>
 </ul>,
 <ol id="ordered_list" style="color:#069">
 <li>Coffee</li>
 <li>Tea</li>
 <li>Milk</li>
 </ol>]

### Accessing attributes

The `ul` tag also contains a `style` attribute. Any bs4 tag behaves like a dictionary with attribute names as keys and attribute values as values:

In [20]:
ulist.attrs

{'id': 'unordered_list', 'style': 'color:#069'}

In [21]:
ulist["style"]

'color:#069'

# Downloading a table from Wikipedia

We aim to get a list of countries sorted by their population size:
https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population

First, let's import the required modules:

In [14]:
import requests
from bs4 import BeautifulSoup
import re
import dateutil

This time, we load the html source directly from a website using the requests module:

In [15]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population")

The web server returns a status code to indicate if the request was (un-)succesfully.
We use that status-code to check if the page was succesfully loaded:

In [16]:
assert result.status_code==200  

Next, we extract the html source and initiated BeautifulSoup:

In [17]:
src = result.content
document = BeautifulSoup(src, 'lxml')

by looking at the document, we can see that we are interested in the first table. So we use `find`:

In [18]:
table = document.find("table")

If you are not familiar with html table, read this example first: https://www.w3schools.com/html/tryit.asp?filename=tryhtml_table_intro

At this point, it is a good idea to programatically check that the table contains the correct header:

In [20]:
assert table.find("th").get_text() == "Rank"

In [27]:
rows = table.find_all("tr")  # Note: this works because find_all is resursive by default


In [29]:
for row in rows[2:-1]:
    cells = row.find_all(["td", "th"])
    
    cells_text = [cell.get_text(strip=True) for cell in cells]
    (rank, country, region, population, percentage, updated_at, source, comment) = cells_text  
    print(f'{rank}, {country}, {region}, {population}')
    

1, China, Asia, 1,412,600,000
2, India, Asia, 1,375,586,000
3, United States, North America, 331,893,745
4, Indonesia, Asia[b], 275,773,800
5, Pakistan, Asia, 229,489,000
6, Nigeria, Africa, 216,747,000
7, Brazil, South America, 215,205,023
8, Bangladesh, Asia, 165,158,616
9, Russia, Europe[c], 145,100,000
10, Mexico, North America, 128,533,664
11, Japan, Asia, 125,927,902
12, Philippines, Asia, 112,489,365
13, Ethiopia, Africa, 105,163,988
14, Egypt, Africa[b], 103,891,636
15, Vietnam, Asia, 98,506,193
16, DR Congo, Africa, 95,241,000
17, Iran, Asia, 85,816,454
18, Turkey, Asia[b], 84,680,273
19, Germany, Europe, 84,079,811
20, France, Europe, 67,918,000
21, United Kingdom, Europe, 67,081,234
22, Thailand, Asia, 66,841,617
23, Tanzania, Africa, 61,280,743
24, South Africa, Africa, 60,604,992
25, Italy, Europe, 58,867,239
26, Myanmar, Asia, 55,294,979
27, South Korea, Asia, 51,638,809
28, Colombia, South America, 51,049,498
29, Kenya, Africa, 47,564,296
30, Spain, Europe, 47,432,805
31

**Attention**: Beautiful Soup does not execute Javascript. This means that you the code in the Google Chrome inspector might look different to the original source code. 

# Another example of downloading a Wikipedia table 

Let's consider another table in a Wikipedia page. This page has a lot more tables, so one challenge will be to pick the right table

https://en.wikipedia.org/wiki/Tiger_Woods


We are interested in extracting these two tables:

![Target Wikipedia tables](pictures/wiki_tables.png)

**Exercise**: 

1) Identify the id="The_Players_Championship", by using title = document.find(id="The_Players_Championship")

2) First find all tables below the id in 1) by title.find_all_next('table').

3) Search for headers (th) by table.find('th') for table in tables to identify the "Tournament" header. Remember to use get_text(strip=True)

4) Save all tables with the header "Tournament" into a list tournament_tables. Check the length of the table and reduce it if it is needed.

5) Bonus: Print out the information in the two tables of interest in the terminal

We begin by downloading the webpage and instatiating the BeautifulSoup object:

In [30]:
result = requests.get("https://en.wikipedia.org/wiki/Tiger_Woods")
src = result.content
document = BeautifulSoup(src, 'lxml')

This page contains a lot of tables without specific attributes that would make it difficult to find our table of interest. Further, the same headings of the tables are used for multiple tables, making it difficult to find a table just by its headings:

In [31]:
len(document.find_all("table"))

60

Therefore, we choose another strategy. First, we extract the tag that defines the header just before our tables of interest. That header tag has a unique identifier attribute `id="The_Players_Championship"`. Then we use the `find_all_next` function in BeautifulSoup to extract all following table tags:

In [32]:
title = document.find(id="The_Players_Championship")
tables = title.find_all_next("table")

Now, our tables of interest are the first two tables with the "Tournament" heading. We write a small helper function (a generator https://wiki.python.org/moin/Generators) that returns a table with a given heading:

In [14]:
def find_table_with_heading(tables, heading):
    for table in tables:
        if table.find("th").get_text(strip=True) == heading:
            yield table

Next, we can extract the table rows and columns as usual. We only extract the first two tables, as these are the only ones we were interested in:

In [16]:
tournament_tables = list(find_table_with_heading(tables, "Tournament"))

for table in tournament_tables[:2]:
    for row in table.find_all("tr"):
        cells= row.find_all(["th", "td"])
        print([cell.get_text(strip="True") for cell in cells])
        

['Tournament', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009']
['The Players Championship', 'T31', 'T35', 'T10', '2', '1', 'T14', 'T11', 'T16', 'T53', 'T22', 'T37', '', '8']
['Tournament', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
['The Players Championship', 'WD', 'WD', 'T40', '1', '', 'T69', '', '', 'T11', 'T30']
